In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

path = ''
df_01 = pd.read_csv(path+"df_01.csv")
df_02 = pd.read_csv(path+"df_02.csv")
df_03 = pd.read_csv(path+"df_03.csv")
df_04 = pd.read_csv(path+"df_04.csv")
df_05 = pd.read_csv(path+"df_05.csv")
df_06 = pd.read_csv(path+"df_06.csv")
df_07 = pd.read_csv(path+"df_07.csv")
df_09 = pd.read_csv(path+"df_09.csv")

name_df = ['df_01','df_02','df_03','df_04','df_05','df_06','df_07','df_09']
list_df = [df_01,df_02,df_03,df_04,df_05,df_06,df_07,df_09]
#make variables "weekend" from 'weekday' (1 if saturday/sunday) and 'daytime' from start_hour (1 if 20-05)
def weekend_daytime(df):
  df['weekend'] = df['weekday'].isin(['Saturday','Sunday']).astype(int)
  df['daytime'] = df['start_hour'].isin([6,7,8,9,10,11,12,13,14,15,16,17,18,19]).astype(int)
  return df
for i in list_df:
  i = weekend_daytime(i)

merge = pd.concat(list_df);merge=merge.dropna(subset=['laf50_per_hour']); merge.reset_index() #drop NAs in the 'laf50_per_hour'
#keep in mind that operations done on train do not apply for the constituent dataframes.
#convert 'date' and 'day' to datetime for both train and test
merge['date'] = pd.to_datetime(merge['date']); merge['day'] = pd.to_datetime(merge['day'])

In [ ]:
# Create an empty dictionary to store accuracy scores
accuracy_scores = {}
predictor_vars = ['LC_HUMIDITY','LC_RAININ','LC_TEMP_QCL3_list','school','loko','depot','weekend','daytime']
data = [df_01,df_02,df_03,df_04,df_05,df_06,df_07,df_09]
name_df = ['df_01','df_02','df_03','df_04','df_05','df_06','df_07','df_09']
param_grid_final = {
    'criterion': ['entropy'],
    'max_depth': [25],
    'min_samples_leaf': [6],
    'max_leaf_nodes': [11],
    'n_estimators': [800],
    'max_samples': [0.6]
}
iter_c = 0
for iter in data:
  data_it = pd.DataFrame(iter)
  data_it = data_it.dropna(subset=['laf50_per_hour'])
  data_it['date'] = pd.to_datetime(data_it['date'])
  X = data_it.loc[:,predictor_vars]
  Y = data_it.loc[:,'noisiness']

  X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=19)
  rf_model = RandomForestClassifier(criterion='entropy',max_depth=25,min_samples_leaf=6,
                                    max_leaf_nodes=6, n_estimators=800,max_samples=0.6,
                                    random_state=19)
  rf_model.fit(X_train, y_train)
  y_pred_rf = rf_model.predict(X_test)
  accuracy_rf = accuracy_score(y_test, y_pred_rf)
  accuracy_scores[name_df[iter_c]] = {
        'Random Forest Accuracy': accuracy_rf}
  joblib.dump(value=rf_model,filename='model_'+str(name_df[iter_c])+'.gz',compress=3)
  iter_c = iter_c+1
#loop ends
accuracy_df = pd.DataFrame.from_dict(accuracy_scores, orient='index')
print(accuracy_df)

       Random Forest Accuracy
df_01                0.634007
df_02                0.627676
df_03                0.645937
df_04                0.669091
df_05                0.640036
df_06                0.620971
df_07                0.621784
df_09                0.635589
